In [ ]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score

s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the data-file
heart = pd.read_csv(file_content_stream)
heart.head()

In [ ]:
## Removing observations with NA
heart = heart.dropna()

In [ ]:
## Defining the input and target variables
X = heart[['male', 'age', 'currentSmoker', 'totChol', 'BMI', 'heartRate', 'sysBP', 'glucose']]
Y = heart['TenYearCHD']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [ ]:
Y_train.value_counts() / Y_train.shape[0]

In [ ]:
Y_test.value_counts() / Y_test.shape[0]

In [ ]:
## Scaling the data 
scaler = MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

# Logistic Regression

In [ ]:
## Indentifying important variables with lasso
lasso_md = LassoCV().fit(X_train, Y_train)

## Extracting the best lambda
lasso_lambda = lasso_md.alpha_

## Fitting LASSO with best lambda
lasso_md = Lasso(alpha = lasso_lambda).fit(X_train, Y_train)
lasso_md.coef_

In [ ]:
## Defining the input and target variables for logistic
X_train_logit = X_train[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]
X_test_logit = X_test[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]

## Building the logistic regression model 
logit_md = LogisticRegression().fit(X_train_logit, Y_train)

## Predicting on test
logit_pred = logit_md.predict_proba(X_test_logit)[:, 1]

## Changing likelihoods to labels
logit_label = np.where(logit_pred < 0.1, 0, 1)

## Computing the recall
recall_score(Y_test, logit_label)

# Random Forest

In [ ]:
## Building the model
RF_md = RandomForestClassifier(n_estimators = 500).fit(X_train, Y_train)

## Extracting the feature importances
importances = pd.DataFrame({'Feature': X_train.columns, 'Importance': RF_md.feature_importances_})
importances = importances.sort_values(by = 'Importance', ascending = False)
importances

In [ ]:
## Defining the input and target variables for logistic
X_train_RF = X_train[['sysBP', 'BMI', 'totChol', 'age', 'glucose']]
X_test_RF = X_test[['sysBP', 'BMI', 'totChol', 'age', 'glucose']]

## Building the random forest model with top 5 featues
RF_md = RandomForestClassifier(n_estimators = 500).fit(X_train_RF, Y_train)

## Predicting on test
RF_pred = RF_md.predict_proba(X_test_RF)[:, 1]

## Changing likelihoods to labels
RF_label = np.where(RF_pred < 0.1, 0, 1)

## Computing the recall
recall_score(Y_test, RF_label)